In [156]:
import cv2
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
from torchvision import models
from PIL import Image

In [157]:
##1.Schritt einbringen von dem 3. Frame von MOT16-04 

img = Image.open("TestFrame000003.jpg")

In [158]:
##2.Schritt einbringen von Koordinaten von einer Results Datei

open("Koordinaten.txt", "r")
Koordinaten = open("Koordinaten.txt", "r")

    ##results = np.loadtxt(Koordinaten, delimiter=',')

    ##nur Zeilen dir mit 3 Starten

    ##alles außer Koordinaten löschen: ..., ..., x, y, w, h, ..., ..., ...
    
    
    ##Koordinaten in Array

In [159]:
##3.Schritt ausschneiden einer Bounding Box von einem dieser Frames
    
x=1362
y=568
w=103
h=241
    
img2 = img.crop((x, y, x+w, y+h))
img2.save('TestBoundingBox.png')

In [160]:
##4.Schritt Transformieren und Normalisieren des Bounding-Box Bildes

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

F.normalize





<function torchvision.transforms.functional.normalize(tensor: torch.Tensor, mean: List[float], std: List[float], inplace: bool = False) -> torch.Tensor>

In [161]:
##5.Schritt ResNet34 Model laden
cnn = torchvision.models.resnet34(pretrained=True)
cnn = torch.nn.Sequential(*(list(cnn.children())[:-1])) 

In [136]:
##6.Schritt Bounding Box Bild durch das CNN jagen